In [23]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [24]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [25]:
NUM_CLASSES = 7

# Dataset reading

In [26]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [27]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [29]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [30]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 42)                0         
                                                                 
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 7)                 77        
                                                                 
Total params: 1147 (4.48 KB)
Trainable params: 1147 (4.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [32]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [33]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/13 [=>............................] - ETA: 11s - loss: 2.0330 - accuracy: 0.0703
Epoch 1: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
13/13 [==============================] - 1s 26ms/step - loss: 1.9372 - accuracy: 0.1038 - val_loss: 1.8271 - val_accuracy: 0.2830
Epoch 2/1000
 1/13 [=>............................] - ETA: 0s - loss: 1.8519 - accuracy: 0.1250
Epoch 2: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
13/13 [==============================] - 0s 8ms/step - loss: 1.7972 - accuracy: 0.2145 - val_loss: 1.6876 - val_accuracy: 0.4830
Epoch 3/1000
 1/13 [=>............................] - ETA: 0s - loss: 1.7198 - accuracy: 0.3047
Epoch 3: saving model to model/keypoint_classifier\keypoint_classifier.hdf5


c:\Users\Aadi\OneDrive\Desktop\TFODCourse\tfod\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 0s 6ms/step - loss: 1.6831 - accuracy: 0.3428 - val_loss: 1.5507 - val_accuracy: 0.5340
Epoch 4/1000
 1/13 [=>............................] - ETA: 0s - loss: 1.6073 - accuracy: 0.3516
Epoch 4: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
13/13 [==============================] - 0s 7ms/step - loss: 1.5625 - accuracy: 0.4340 - val_loss: 1.4349 - val_accuracy: 0.4906
Epoch 5/1000
 1/13 [=>............................] - ETA: 0s - loss: 1.4563 - accuracy: 0.4922
Epoch 5: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
13/13 [==============================] - 0s 6ms/step - loss: 1.4821 - accuracy: 0.4623 - val_loss: 1.3663 - val_accuracy: 0.4906
Epoch 6/1000
 1/13 [=>............................] - ETA: 0s - loss: 1.4477 - accuracy: 0.4375
Epoch 6: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
13/13 [==============================] - 0s 6ms/step - loss: 1.4166 - accuracy: 0.4792 - val_loss

In [34]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

5/5 [==============================] - 0s 2ms/step - loss: 0.2755 - accuracy: 0.9358


In [35]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [36]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 76ms/step
[6.5331440e-04 9.8698390e-01 2.0456837e-05 1.1288341e-02 1.0540213e-03
 8.4941103e-09 2.4598727e-09]
1


# Confusion matrix

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

# Convert to model for Tensorflow-Lite

In [38]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [39]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Aadi\AppData\Local\Temp\tmpawlxlj39\assets


INFO:tensorflow:Assets written to: C:\Users\Aadi\AppData\Local\Temp\tmpawlxlj39\assets


6704

# Inference test

In [40]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [41]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [42]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [43]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [44]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[6.5331440e-04 9.8698390e-01 2.0456837e-05 1.1288347e-02 1.0540212e-03
 8.4941103e-09 2.4598725e-09]
1
